In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from sklearn import preprocessing as pp
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
df_w5 = pd.read_stata('/Users/david.yan/Downloads/HCMST_wave_5_supplement_ver_1.dta')
df_w4 = pd.read_stata('/Users/david.yan/Downloads/wave_4_supplement_v1_2.dta')
df_w123 = pd.read_stata('/Users/david.yan/Downloads/HCMST_ver_3.04.dta')

In [3]:
w5_cols = df_w5.columns.tolist()
w4_cols = df_w4.columns.tolist()
w123_cols = df_w123.columns.tolist()

In [4]:
print(len(w5_cols), len(w4_cols), len(w123_cols))

(78, 62, 387)


In [5]:
print(df_w123.shape, df_w4.shape, df_w5.shape)

((4002, 387), (4002, 62), (4002, 78))


In [6]:
wave_1_couples = df_w123.loc[(df_w123['qflag'] == 'partnered') & (df_w123['partner_deceased'] == 'not deceased'), 'caseid_new':]

In [7]:
wave_1_couples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2981 entries, 0 to 4001
Columns: 387 entries, caseid_new to w3_nonmbtiming_month
dtypes: category(325), float32(16), float64(39), int32(4), int8(2), object(1)
memory usage: 2.1+ MB


In [8]:
wave_1_couples['pp3_newer'].value_counts()

Yes, pp3 data is newer and available    2023
no, newer pp3 data is Not available      958
Name: pp3_newer, dtype: int64

In [51]:
# Get relationship status of couples that have deceased partners
w2_dead = wave_1_couples[wave_1_couples.w2_broke_up == 'partner passed away'].index
w3_dead = wave_1_couples[wave_1_couples.w3_broke_up == 'partner deceased'].index
w4_dead = df_w4[df_w4.w4_broke_up == 'partner passed away'].index
w5_dead = df_w5[df_w5.w5_broke_up == 'partner deceased'].index

print len(wave_1_couples.loc[wave_1_couples.loc[w2_dead,['qflag']].index][wave_1_couples.qflag == 'partnered'].index)
print ''
print len(wave_1_couples.loc[wave_1_couples.loc[w3_dead,['w2_broke_up']].index][wave_1_couples.w2_broke_up == 'still together'].index)
print ''
print len(wave_1_couples.loc[wave_1_couples.loc[w4_dead,['w3_broke_up']].index][wave_1_couples.w3_broke_up == 'still together'].index)
print ''
print len(df_w4.loc[df_w4.loc[w5_dead,['w4_broke_up']].index][df_w4.w4_broke_up == 'still together'].index)

/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


40

18

12

8


/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [57]:
w2_dead_st = wave_1_couples.loc[wave_1_couples.loc[w2_dead,['qflag','caseid_new']].index][wave_1_couples.qflag == 'partnered'].caseid_new.values
w3_dead_st = wave_1_couples.loc[wave_1_couples.loc[w3_dead,['w2_broke_up','caseid_new']].index][wave_1_couples.w2_broke_up == 'still together'].caseid_new.values
w4_dead_st = wave_1_couples.loc[wave_1_couples.loc[w4_dead,['w3_broke_up','caseid_new']].index][wave_1_couples.w3_broke_up == 'still together'].caseid_new.values
w5_dead_st = df_w4.loc[df_w4.loc[w5_dead,['w4_broke_up','caseid_new']].index][df_w4.w4_broke_up == 'still together'].caseid_new.values

/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/david.yan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [64]:
# prior relationship status of all couples that have deceased partners is still together
tot_dead_list = np.concatenate((w2_dead_st,w3_dead_st,w4_dead_st,w5_dead_st))

In [65]:
wave_2_couples_broke_up = wave_1_couples.loc[wave_1_couples['w2_broke_up']=='broke up']
wave_3_couples_broke_up = wave_1_couples.loc[wave_1_couples['w3_broke_up']=='broke up']
w4_breakup = df_w4.loc[df_w4['w4_broke_up']=='broke up']
w5_breakup = df_w5.loc[df_w5['w5_broke_up']=='broke up']
survived_couples=df_w5.loc[df_w5['w5_broke_up']=='still together']

In [66]:
w2_breakup_list=wave_2_couples_broke_up['caseid_new'].tolist()
w3_breakup_list=wave_3_couples_broke_up['caseid_new'].tolist()
w4_breakup_list=w4_breakup['caseid_new'].tolist()
w5_breakup_list=w5_breakup['caseid_new'].tolist()
survived_couples_list=survived_couples['caseid_new'].tolist()

In [67]:
for x in tot_dead_st:
    if x in w2_breakup_list or x in w3_breakup_list or x in w4_breakup_list or x in w5_breakup_list:
        print(x)
# print(len(w5_breakup_list)+len(w4_breakup_list)+len(w3_breakup_list)+len(w2_breakup_list))
# print(len(survived_couples_list))